In [47]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from datetime import datetime

In [48]:
region = boto3.Session().region_name
session = sagemaker.Session()

BUCKET = 'covid-19-image-dataset'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)
s3 = boto3.resource('s3')

In [49]:
from sagemaker import get_execution_role
role = get_execution_role() 
print (role)

arn:aws:iam::154108359553:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


## Dataset Information
Helping Deep Learning and AI Enthusiasts like me to contribute to improving COVID-19 detection using just Chest X-rays. It is a simple directory structure branched into test and train and further branched into the respective 3 classes which contains the images. It contains around 137 cleaned images of COVID-19 and 317 in total containing Viral Pneumonia and Normal Chest X-Rays structured into the test and train directories.


In [50]:
train_data_s3 = 's3://{}/{}'.format(BUCKET, 'training_data')

## SageMaker Training

In [51]:
src_code_s3 = 's3://{}/{}'.format(BUCKET, 'training_jobs')
training_job_output_s3 = 's3://{}/{}'.format(BUCKET, 'training_jobs_output')

In [52]:
hp = {'epochs': 2,
      'backend': 'nccl',
      'lr': 0.001,
      'batch-size': 8
}

In [53]:
# Encapsulate training on SageMaker with PyTorch:
train_estimator = PyTorch(entry_point='transfer_learning_resnet.py',
                          role=role,
                          framework_version='1.8.1',
                          py_version='py3',
                          debugger_hook_config=False,
                          instance_count=1,
                          instance_type='ml.p3.2xlarge',
                          output_path=training_job_output_s3,
                          code_location=src_code_s3,
                          hyperparameters=hp
)

In [54]:
# Setting up File-system to import data from S3
data_channels = {'train': sagemaker.inputs.TrainingInput(
                          s3_data_type='S3Prefix',
                          s3_data=train_data_s3,
                          content_type='image/jpeg',
                          input_mode='File'),
                 
                 'val': sagemaker.inputs.TrainingInput(
                        s3_data_type='S3Prefix',
                        s3_data=train_data_s3,
                        content_type='image/jpeg',
                        input_mode='File')
}

In [55]:
# Launching SageMaker training job
train_job_id = 'sm-jam-covid-' + str(datetime.now().strftime("%H-%M-%S"))
train_estimator.fit(inputs=data_channels, job_name=train_job_id)

2021-10-07 09:24:18 Starting - Starting the training job...
2021-10-07 09:24:46 Starting - Launching requested ML instancesProfilerReport-1633598657: InProgress
......
2021-10-07 09:25:46 Starting - Preparing the instances for training......
2021-10-07 09:26:46 Downloading - Downloading input data...
2021-10-07 09:27:07 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-07 09:30:56,217 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-07 09:30:56,242 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-07 09:30:56,867 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-07 09:30:57,262 sagemaker-training-toolkit INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
   

## TODO: SageMaker Inference